In [1]:
import numpy as np
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)


#bring in dipoles and electronic energies
dipoles = np.load("LiH_r_1.4_6311g_fci_Dipoles.npy")
electronic_energies = np.load("LiH_r_1.4_6311g_fci_Energies.npy")

# number of electronic states
N_el = len(electronic_energies)

# lambda vector in atomic units - set this as a parameter!
lambda_vec = np.array([0.0, 0.0, 0.05])

# photon frequency in atomic units
omega = 0.12086 

# vector potential in atomic units - determined by magnitude of \lambda
A0 = np.sqrt(1/(2 * omega)) * lambda_vec

# size of photon basis - 2 means |0> and |1> photon number states
Np = 2

# size of composite energy array
N_dim = Np * N_el


# build an array of total energies depending on how many photon number states 
# we want to include 
composite_energies = np.zeros(N_dim)

# here we will store electronic and photonic indices as tuples in case these are needed
composite_indices = []

# order them as E_0, E_1, E_2, ... , E_N_el, E_0 + \omega, E_1 + \omega, ..., E_0 + (Np - 1) \omega, E_1 + (Np - 1) \omega
# can do this with slices but loop is ok for now
for i in range(Np):
    for j in range(N_el):
        composite_energies[i*N_el + j] = electronic_energies[j] + i * omega
        composite_indices.append((j,i))


# just demonstrate the storage convention
print(F'For composite energy {53} with value {composite_energies[53]}')
print(F'The electronic index is {composite_indices[53][0]} and the photonic index is {composite_indices[53][1]}')
print(F'Which is the same as {electronic_energies[composite_indices[53][0]]} + {omega * composite_indices[53][1]}')

m_n = 0 #num_photons
mu_n = 0 #electronic state

print(len(composite_indices))




For composite energy 53 with value -7.740188629865463
The electronic index is 3 and the photonic index is 1
Which is the same as -7.861048629865463 + 0.12086
100


\begin{equation}
        \begin{split}
            {E}_{\rm n}^{(2)}
            &= \omega^{2} \sum_{l\neq n}\frac{| d_{{\mu_l}{\mu_n}}\sqrt{m_n + 1}\delta_{{m_l}{m_{n}+1}} |^{2}}{{E}_{\mu_n} - {E}_{\mu_l} -  \omega \hbar} \\
            &\qquad+\frac{| d_{{\mu_l}{\mu_n}}\sqrt{m_n}\delta_{{m_l}{m_{n}-1}} |^{2}}{{E}_{\mu_n} - {E}_{\mu_l} +  \omega \hbar} \\
            &\qquad\qquad 
            +\frac{ |\sum_{\gamma} d_{\mu_{l} \gamma} d_{\gamma \mu_{n}} \delta_{ m_{l}m_{n}}|^{2}}{{E}_{\mu_n}  - {E}_{\mu_l}}
            \\
        \end{split}
    \end{equation}

In [2]:

def build_d_matrix(A_vec, dipoles):
    """
    form d by dotting the A vector into the dipole matrix elements
    """
    
    return np.einsum('k, ijk -> ij', A_vec, dipoles)

def j_fo(N, d, om):
    """
    Compute first order energy correction.
    Sum only runs over electronic states because the bra and ket must have the same photon index
    
    Arguments
    ---------
    N : int 
        The composite index of the state for which we are computing the correction
        
    d : (N_el, N_el) array of floats
        The array of dipole matrix elements dotted into the field vector
        
    om : float
        The photon frequency
        
    Returns
    -------
    En_1 : float
        The first order energy correction
    """
    
    # compute the contraction over the d matrix
    t1 = np.einsum("ij,jk", d, d)
    
    # compute the first order energy correction
    En_1 = t1[N,N] * om
    
    # return the first order energy correction
    return En_1
    

def j_so(N, En, d, om, Co):
    """ 
    Compute the second order energy correction.
    
    Arguments
    ---------
    N : int
        The composite index of the state for which we are computing the correction
        
    En : (N_el,) array of floats
        The electronic energies for each electronic state
        
    d : (N_el, N_el) array of floats
        The array of dipole matrix elements dotted into the field vector
        
    om : float
        The photon frequency
        
    Co : (N_el * Np) array of tuples
        The tuples contain the electronic and photonic indices for composite index of state N
        
    Returns
    -------
    En_2 : float
         The second order energy correction
    """
    
    # get the total number of states
    N_max = len(Co)
    
    # get the total number of electronic states
    g_max = len(En)
    
    # photon index for state N
    m_n = Co[N][1]
    
    # electronic index for state N
    mu_n = Co[N][0]
    
    t_1 = 0
    t_2 = 0
    t_3 = 0
    for l in range(N_max):
        if l != N:
            mu_l = Co[l][0]
            m_l = Co[l][1]
            
            if m_l == (m_n + 1):
                num = d[mu_l, mu_n] * np.sqrt(m_n + 1)
                t_1 += num ** 2 / (En[mu_n] - En[mu_l] - om)
                
            
            elif m_l == (m_n - 1):
                num = d[mu_l, mu_n] * np.sqrt(m_n)
                t_2 += num ** 2 / (En[mu_n] - En[mu_l] + om)
                
            elif m_l == m_n:
                
                t3_es = np.einsum("lg,gn", d, d)
                t_3 += t3_es[mu_n, mu_l] ** 2 / (En[mu_n] - En[mu_l])
                    
                #t_3 += num ** 2 / (En[mu_n] - En[mu_l])
                
    En_2 = om ** 2 * (t_1 + t_2 + t_3)
    print(F'First term is {t_1}')
    print(F'Second term is {t_2}')
    print(F'Third term is {t_3}')
    print(F'En_2 is {En_2}')
    
    return En_2
                
            
            
            
            





d_matrix = build_d_matrix(A0, dipoles)






In [3]:



def calc_first_order_energy_correction(N_el, d_matrix, omega, mu_n):

    E_n_1 = 0
    for gamma in range(0, N_el):
        E_n_1 += d_matrix[mu_n][gamma]*d_matrix[gamma][mu_n]

    E_n_1 = omega* E_n_1

    return E_n_1                




def delta(i,j):
    if i ==j:
        return 1
    else:
        return 0

#unsimplified version of second order energy correction to check if the same answer is gotten
def calc_second_order_energy_correction_unsimplified(d_matrix, omega, mu_n, energies, m_n):
    E_n_2 = 0


    for mu_l in range(0, len(energies)):

            #for m_l in range(max(m_n-1,0), m_n+2):
            for m_l in range(max(0, m_n-1), m_n+2 ):

                if(mu_l == mu_n and m_l == m_n):
                    pass
                else:

                    sum_over_energy_states_gamma = 0
                    for gamma in range(0, len(energies)):
                        #numerator += d_matrix[gamma][mu_l]*d_matrix[mu_l][gamma]
                        sum_over_energy_states_gamma += (d_matrix[mu_l][gamma]*d_matrix[gamma][mu_n] * delta(m_l, m_n))

                    
                    E_n_2 += (((-omega * d_matrix[mu_l][mu_n] * (np.sqrt(m_n +1) * delta(m_l, m_n + 1)  + np.sqrt(m_n) * delta(m_l, m_n-1)) ) + (omega * sum_over_energy_states_gamma))**2   )/ (energies[mu_n] + (m_n * omega) - energies[mu_l] - (m_l * omega))

                    #print((((-omega * d_matrix[mu_l][mu_n] * (np.sqrt(m_n +1) * delta(m_l, m_n +1)  + np.sqrt(m_n) * delta(m_l, m_n-1)) ) + (omega * sum_over_energy_states_gamma))**2   )/ (energies[mu_n] + (m_n * omega) - energies[mu_l] - (m_l * omega)))


    return E_n_2




def calc_second_order_energy_correction(d_matrix, omega, mu_n, energies, m_n):
    E_n_2 = 0


    for mu_l in range(0, len(energies)):
            for m_l in range(max(m_n-1,0), m_n+2):
                if(mu_l == mu_n and m_l == m_n):
                    pass
                else:

                    if m_l == m_n + 1:
                        E_n_2 +=  ( (d_matrix[mu_l][mu_n] * np.sqrt(m_n+1)) ** 2 )/(energies[mu_n] - energies[mu_l] - omega)

                        #print( d_matrix[mu_l][mu_n]  )


                    elif m_l  == m_n - 1:
                        E_n_2 +=  ( (d_matrix[mu_l][mu_n] * np.sqrt(m_n)) ** 2 )/(energies[mu_n] - energies[mu_l] + omega)

                        #print(( (d_matrix[mu_l][mu_n] * np.sqrt(m_n+1)) ** 2 )/(energies[mu_n] - energies[mu_l] - omega))

        
                    elif m_l  == m_n:

                        
                        numerator = 0
                        for gamma in range(0, len(energies)):
                            numerator += d_matrix[mu_l][gamma]*d_matrix[gamma][mu_n]

                            #print("numerator: ", numerator)


                            #print("d_matrix[mu_l][gamma]: ",  d_matrix[mu_l][gamma] ," d_matrix[gamma][mu_n]: ", d_matrix[gamma][mu_n])
                

                        E_n_2 += (numerator**2) / ((energies[mu_n] - energies[mu_l]))

                        #print("denominator:" , ((energies[mu_n] - energies[mu_l])))

                        #print("contrib: ",(numerator**2) / ((energies[mu_n] - energies[mu_l])) )



    E_n_2 = E_n_2 * omega**2
    return E_n_2




def calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2):
    return electronic_energies[mu_n] + (omega * m_n) + E_n_1 + E_n_2 

def calc_energy_correction_to_third_order(mu_n, E_n_1, E_n_2, E_n_3):
    return electronic_energies[mu_n] + (omega * m_n) + E_n_1 + E_n_2 + E_n_3




E_n_1 = calc_first_order_energy_correction(N_el, d_matrix, omega, mu_n)


E_n_2 = calc_second_order_energy_correction(d_matrix, omega, mu_n, electronic_energies, m_n=m_n)


out1 = j_fo(0, d_matrix, omega)
out2 = j_so(0, electronic_energies, d_matrix, omega, composite_indices)



print("E_n_1: ", E_n_1)
print("E-n_2: ", E_n_2)
assert np.isclose(E_n_1, out1)
assert np.isclose(E_n_2, out2)


print("uncorrected_energy:" , electronic_energies[mu_n]+(omega * m_n))

print("energy corrected to second order: " , calc_energy_correction_to_second_order(mu_n, E_n_1, E_n_2))

qed_fci_energies = np.load("LiH_r_1.4_6311g_qedfci_Energies.npy")
print("qed_fci_energy:", qed_fci_energies[mu_n])

First term is -0.4495429039440164
Second term is 0
Third term is -0.020198518650927123
En_2 is -0.006861578535746935
E_n_1:  0.008852391285035493
E-n_2:  -0.006861578535746932
uncorrected_energy: -8.012194758186697
energy corrected to second order:  -8.010203945437407
qed_fci_energy: -8.00913664849758
